<a href="https://colab.research.google.com/github/nhAnik/MlProject/blob/master/quoraInsicereQuestions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2019-02-07 04:46:50--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.840B.300d.zip [following]
--2019-02-07 04:46:50--  https://nlp.stanford.edu/data/glove.840B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176768927 (2.0G) [application/zip]
Saving to: ‘glove.840B.300d.zip’

glove.840B.300d.zip 100%[===================>]   2.03G  26.8MB/s    in 92s     

2019-02-07 04:48:22 (22.6 MB/s) - ‘glove.840B.300d.zip’ saved [2176768927/2176768927]



In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import zipfile
zipRef=zipfile.ZipFile('glove.840B.300d.zip','r')
zipRef.extractall()
zipRef.close()

In [0]:
import numpy as np 
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

In [0]:
import pandas as pd
trainDf=pd.read_csv("/content/gdrive/My Drive/colabData/MlProject/train.csv")

In [0]:
def readEmbeddings(embeddingsFile):
    
    tempDict={}
    f=open(embeddingsFile)
    for line in tqdm(f):
        values=line.split(" ")
        word=values[0]
        vect=np.asarray(values[1:],dtype='float32')
        tempDict[word]=vect
    f.close()
   
    return tempDict
embeddingsDict=readEmbeddings('glove.840B.300d.txt')

vectDim=300
numOfVect=30

2196017it [02:50, 12847.31it/s]


In [0]:
trainDf, valDf = train_test_split(trainDf, test_size=0.4)
valDf, testDf = train_test_split(valDf, test_size=0.5)
print(trainDf.shape)
print(testDf.shape)
print(valDf.shape) 

(783673, 3)
(261225, 3)
(261224, 3)


In [0]:
def textToArr(txt):
    zeroVect=np.zeros(vectDim)
    txt=txt[:-1].split()[:numOfVect]
    embeds=[embeddingsDict.get(x,zeroVect) for x in txt]
    embeds+=[zeroVect]*(numOfVect-len(embeds)) 
    return np.array(embeds)

textToArr("what is quora?").shape

(30, 300)

In [0]:
batchSize = 128

def batchGenerate(train_df):
    numBatches = math.ceil(len(train_df) / batchSize)
    while True: 
        train_df = train_df.sample(frac=1.)   
        for i in range(numBatches):
            texts = train_df.iloc[i*batchSize:(i+1)*batchSize, 1]
            textArr = np.array([textToArr(text) for text in texts])
            yield textArr, np.array(train_df["target"][i*batchSize:(i+1)*batchSize])

In [0]:
from keras import backend as K

def recall(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall
      
def precision(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1(y_true, y_pred):
    prec = precision(y_true, y_pred)
    rec = recall(y_true, y_pred)
    return 2*((prec*rec)/(prec+rec+K.epsilon()))

Using TensorFlow backend.


In [16]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Dropout

model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True), input_shape=(30, 300))) 
model.add(Bidirectional(CuDNNLSTM(64)))  
model.add(Dense(1, activation="sigmoid"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy',recall,precision,f1])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_5 (Bidirection (None, 30, 128)           187392    
_________________________________________________________________
bidirectional_6 (Bidirection (None, 128)               99328     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 286,849
Trainable params: 286,849
Non-trainable params: 0
_________________________________________________________________


In [17]:
trainGen = batchGenerate(trainDf)
valGen=batchGenerate(valDf)
model.fit_generator(trainGen, epochs=10, steps_per_epoch=4000, validation_data=valGen, validation_steps=2000, verbose=True) 

Epoch 1/10
4000/4000 [==============================] - 225s 56ms/step - loss: 0.1207 - acc: 0.9529 - recall: 0.4660 - precision: 0.6630 - f1: 0.5213 - val_loss: 0.1121 - val_acc: 0.9553 - val_recall: 0.6014 - val_precision: 0.6499 - val_f1: 0.6081
Epoch 2/10
4000/4000 [==============================] - 224s 56ms/step - loss: 0.1067 - acc: 0.9577 - recall: 0.5512 - precision: 0.6999 - f1: 0.5952 - val_loss: 0.1078 - val_acc: 0.9577 - val_recall: 0.5139 - val_precision: 0.7233 - val_f1: 0.5813
Epoch 3/10
4000/4000 [==============================] - 227s 57ms/step - loss: 0.1022 - acc: 0.9595 - recall: 0.5770 - precision: 0.7133 - f1: 0.6166 - val_loss: 0.1056 - val_acc: 0.9583 - val_recall: 0.5945 - val_precision: 0.6907 - val_f1: 0.6206
Epoch 4/10
4000/4000 [==============================] - 230s 58ms/step - loss: 0.0960 - acc: 0.9616 - recall: 0.6131 - precision: 0.7291 - f1: 0.6474 - val_loss: 0.1046 - val_acc: 0.9586 - val_recall: 0.6282 - val_precision: 0.6808 - val_f1: 0.6367
Epoc

In [18]:
testGen=batchGenerate(testDf)
score = model.evaluate_generator(testGen, steps=2000, verbose=1)

2000/2000 [==============================] - 36s 18ms/step


In [19]:
print(model.metrics_names[0],":",score[0])
print(model.metrics_names[1],":",score[1])
print(model.metrics_names[2],":",score[2])
print(model.metrics_names[3],":",score[3])
print(model.metrics_names[4],":",score[4])

loss : 0.1257426714557223
acc : 0.95458203125
recall : 0.6159595116488635
precision : 0.6395830409601331
f1 : 0.6101039763465523
